In [1]:
from azure.ai.ml import MLClient
from azure.identity import DefaultAzureCredential

# authenticate
credential = DefaultAzureCredential()
SUBSCRIPTION=<subscription id>
RESOURCE_GROUP=<name of resource group>
WS_NAME=<Name of Workspace>

# Get a handle to the workspace
ml_client = MLClient(
    credential=credential,
    subscription_id=SUBSCRIPTION,
    resource_group_name=RESOURCE_GROUP,
    workspace_name=WS_NAME,
)

In [2]:
# Create Endpoint Start

In [3]:
import uuid

# Create a unique name for the endpoint
online_endpoint_name = "improved-img-endpoint-" + str(uuid.uuid4())[:8]

In [4]:
from azure.ai.ml.entities import ManagedOnlineEndpoint

# define an online endpoint
endpoint = ManagedOnlineEndpoint(
    name=online_endpoint_name,
    description="animal img pred SVM",
    auth_mode="key",
)

In [5]:
# create the online endpoint
# expect the endpoint to take approximately 2 minutes.

endpoint = ml_client.online_endpoints.begin_create_or_update(endpoint).result()

In [6]:
endpoint = ml_client.online_endpoints.get(name=online_endpoint_name)

print(
    f'Endpoint "{endpoint.name}" with provisioning state "{endpoint.provisioning_state}" is retrieved'
)

Endpoint "improved-img-endpoint-735caac1" with provisioning state "Succeeded" is retrieved


In [7]:
# Create Deployment start

In [8]:
from azure.ai.ml.entities import ManagedOnlineDeployment

# This is my endpoint name:
online_endpoint_name = "jbwppet-img-endpoint-64ea12ad"

# Choose the latest version of the registered model for deployment
model = ml_client.models.get(name="image_classification_model", version=3)

# define an online deployment
# if you run into an out of quota error, change the instance_type to a comparable VM that is available.\
# Learn more on https://azure.microsoft.com/en-us/pricing/details/machine-learning/.
blue_deployment = ManagedOnlineDeployment(
    name="blue",
    endpoint_name=online_endpoint_name,
    model=model,
    instance_type="Standard_E2ds_v4",
    instance_count=1,
)

In [9]:
# create the online deployment
blue_deployment = ml_client.online_deployments.begin_create_or_update(blue_deployment).result()

Check: endpoint jbwppet-img-endpoint-64ea12ad exists


ResourceNotFoundError: (ResourceNotFound) The Resource 'Microsoft.MachineLearningServices/workspaces/from_model_to_production/onlineEndpoints/jbwppet-img-endpoint-64ea12ad' under resource group 'from_model_to_production' was not found. For more details please go to https://aka.ms/ARMResourceNotFoundFix
Code: ResourceNotFound
Message: The Resource 'Microsoft.MachineLearningServices/workspaces/from_model_to_production/onlineEndpoints/jbwppet-img-endpoint-64ea12ad' under resource group 'from_model_to_production' was not found. For more details please go to https://aka.ms/ARMResourceNotFoundFix

In [ ]:
# assign traffic and creat endpoint
endpoint.traffic = {"blue": 100}
ml_client.online_endpoints.begin_create_or_update(endpoint).result()

In [ ]:
# Check the status of the endpoint

In [9]:
# return an object that contains metadata for the endpoint
endpoint = ml_client.online_endpoints.get(name=online_endpoint_name)

# print a selection of the endpoint's metadata
print(
    f"Name: {endpoint.name}\nStatus: {endpoint.provisioning_state}\nDescription: {endpoint.description}"
)

Name: jbwppet-img-endpoint-64ea12ad
Status: Succeeded
Description: this is an online endpoint


In [10]:
# existing traffic details
print(endpoint.traffic)

# Get the scoring URI
print(endpoint.scoring_uri)

{'image-classification-model-3': 0}
https://jbwppet-img-endpoint-64ea12ad.northeurope.inference.ml.azure.com/score
